In [1]:
from lively import Solver, Translation, Rotation,Transform, OrientationLivelinessObjective, OrientationMatchObjective, SmoothnessMacroObjective, PositionMatchObjective, JointMatchObjective, CollisionAvoidanceObjective, JointLimitsObjective, BoxShape, CollisionSettingInfo, ScalarRange
from lxml import etree

# Constructed CollisionAvoidance Objective ,BoxShape and configured Collision Setting
box = BoxShape(name="Table",frame="world",physical=True,x=2,y=1,z=1.2,translation = Translation(x=1.0, y =2.0, z=3.0),
rotation = Rotation(x=0.0,y=0.0,z=0.0,w=1.0))
collision = CollisionSettingInfo(d_max = 0.1, r = 0.0, a_max = 2.0, time_budget = 100, timed = False)

# Read the xml file into a string
xml_file = '../../tests/panda.xml'
tree = etree.parse(xml_file)
xml_string = etree.tostring(tree).decode()
#print(xml_string)
# Instantiate a new solver
solver = Solver(
  urdf=xml_string, # Full urdf as a string
  objectives={
      # An example objective (smoothness macro)
      "smoothness":SmoothnessMacroObjective(name="MySmoothnessObjective",weight=25,joints=True,origin=False,links=True),
      "collision": CollisionAvoidanceObjective(name="MyCollisionAvoidanceObjective", weight=0.5),
      "jointLimit": JointLimitsObjective(name="MyJointLimitObjective", weight=5),
      "torso Position": PositionMatchObjective(name="MyPositionMatchObjective", weight=10, link="torso"),
      "positionMatch": PositionMatchObjective(name="R Hand Position", weight=10, link="r_gripper"),
      "r hand orientation": OrientationMatchObjective(name="R Hand Orientation", weight=10, link="r_gripper"),
      "headOrientation": OrientationMatchObjective(name="Gaze",link="Head",weight=7),
      "idleGaze": OrientationLivelinessObjective(name="Idle Gaze", link= "Head", weight=20, frequency=10),
      "LShoulderPitch": JointMatchObjective(name="LShoulderPitch", joint="LShoulderPitch", weight=10),
      "LShoulderRoll": JointMatchObjective(name="LShoulderRoll", joint="LShoulderRoll", weight=10),
      "LElbowYaw": JointMatchObjective(name="LElbowYaw", joint="LElbowYaw", weight=10),
      "LElbowRoll": JointMatchObjective(name="LElbowRoll", joint="LElbowRoll", weight=10),
      "LWristYaw": JointMatchObjective(name="LWristYaw", joint="LWristYaw", weight=10),
      "LHand": JointMatchObjective(name="LHand", joint="LHand", weight=10),
      
  },
  root_bounds=[
      ScalarRange(value=0.0,delta=0.0),ScalarRange(value=0.0,delta=0.0),ScalarRange(value=0.0,delta=0.0), # Translational, (x, y, z)
      ScalarRange(value=0.0,delta=0.0),ScalarRange(value=0.0,delta=0.0),ScalarRange(value=0.0,delta=0.0)  # Rotational, (r, p, y)
  ],
  shapes=[
      box
  ],
  collision_settings = collision

)

# Run solve to get a solved state
state = solver.solve(goals= {"MyGoal1": Translation(x=1.0, y =2.0, z=3.0)},weights = {},time = 0.0)
# Log the initial state
print(state.origin.as_dicts())
print(state.joints)
print(state.frames["panda_link0"])
print(state.proximity)

({'x': 0.0, 'y': 0.0, 'z': 0.0}, {'z': 0.0, 'y': 0.0, 'x': 0.0, 'w': 1.0})
{'panda_joint1': 9.948786930518066e-13, 'panda_joint7': 4.581040777949828e-07, 'panda_joint2': 9.928497154444936e-13, 'panda_joint6': 1.8675023256031476, 'panda_finger_joint1': 0.020000478122966445, 'panda_finger_joint2': 0.020000478122966445, 'panda_joint5': 4.5810414785932124e-07, 'panda_joint3': 4.581041437981835e-07, 'panda_joint4': -1.5707995418958756}
TransformInfo: world:{translation: {x:0.0, y:0.0, z:0.0}, rotation: {w:1.0, x:0.0, y:0.0, z:0.0}}, local: {translation: {x:0.0, y:0.0, z:0.0}, rotation: {w:1.0, x:0.0, y:0.0, z:0.0}}
[ProximityInfo: {shape1: "panda_link1", shape2: "panda_link2", distance: -0.04999999999998844, points: Some(([-5.170949430553605e-9, -7.786133364021042e-14, 0.3929999999999937], [5.171018930033686e-9, -8.176966048711876e-14, 0.3430000000000063])), physical: true, loss: 1.0499999999999885, average_distance: Some(1.0)}, ProximityInfo: {shape1: "panda_link3", shape2: "panda_link5", 